# Setup

### Libraries and Packages

In [2]:
import pandas as pd
import datetime
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### Import Data


In [15]:
# ACLED Ethiopia - for conflict analysis
eth = pd.read_csv('/content/ethiopia.csv',',')

# ACLED Ukraine - for conflict analysis
ukr = pd.read_csv('/content/ukraine.csv',',')

# NYT Metadata - for news trend analysis
nyt = pd.read_csv('/content/nyt.csv',',')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
eth

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,9610112,231,ETH9332,9332,04 November 2022,2022,1,Battles,Armed clash,OLF: Oromo Liberation Front (Shane Splinter Fa...,...,Begi,9.3457,34.5315,2,Oromiya Media Network,International,"On 4 November 2022, OLF Shane clashed with END...",0,1667850819,ETH
1,9610127,231,ETH9333,9333,04 November 2022,2022,1,Battles,Armed clash,OLF: Oromo Liberation Front (Shane Splinter Fa...,...,Nejo,9.5000,35.5000,2,Oromiya Media Network,International,"On 4 November 2022, and for the third consecut...",0,1667850819,ETH
2,9610122,231,ETH9331,9331,03 November 2022,2022,1,Battles,Armed clash,OLF: Oromo Liberation Front (Shane Splinter Fa...,...,Nejo,9.5000,35.5000,2,Oromiya Media Network,International,"On 3 November 2022, and for the second consecu...",0,1667850819,ETH
3,9610114,231,ETH9330,9330,02 November 2022,2022,1,Battles,Armed clash,OLF: Oromo Liberation Front (Shane Splinter Fa...,...,Nejo,9.5000,35.5000,2,Oromiya Media Network,International,"On 2 November 2022, OLF Shane clashed with END...",0,1667850819,ETH
4,9610115,231,ETH9329,9329,02 November 2022,2022,1,Explosions/Remote violence,Air/drone strike,Military Forces of Ethiopia (2018-),...,Bila,9.3693,35.5986,1,Oromiya Media Network,International,"On 2 November 2022, Ethiopian Airforce conduct...",50,1667850819,ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,8696243,231,ETH5725,5725,05 November 2020,2020,1,Battles,Armed clash,TPLF: Tigray People's Liberation Front,...,Soroca,13.6167,36.5333,1,BBC News,International,"On 5 November 2020, TPLF forces reportedly att...",6,1639526080,ETH
3562,8696244,231,ETH5723,5723,05 November 2020,2020,1,Battles,Armed clash,TPLF: Tigray People's Liberation Front,...,Abderafi,13.7333,36.4500,1,Reuters,International,"On 5 November 2020, for the second day, Ethiop...",0,1639526080,ETH
3563,8696245,231,ETH5790,5790,04 November 2020,2020,1,Battles,Non-state actor overtakes territory,TPLF: Tigray People's Liberation Front,...,Adigrat,14.2770,39.4620,2,BBC News,International,"On 4 November 2020, TPLF forces and associate ...",66,1639526080,ETH
3564,8696246,231,ETH5722,5722,04 November 2020,2020,1,Battles,Armed clash,TPLF: Tigray People's Liberation Front,...,Abderafi,13.7333,36.4500,2,BBC News,International,"On 4 November 2020, Ethiopian military forces ...",0,1639526080,ETH


### Formatting

In [57]:
# Dates

eth['event_date'] = pd.to_datetime(eth['event_date'],format = '%d %B %Y')
eth['year'] = pd.to_datetime(eth['year'], format = '%Y')
ukr['event_date'] = pd.to_datetime(ukr['event_date'],format = '%d %B %Y')
ukr['year'] = pd.to_datetime(ukr['year'], format = '%Y')
nyt['date'] = pd.to_datetime(nyt['date'],format = '%Y-%m-%d')
nyt['headline'] = nyt['headline'].to_string()

# IDs

convert_dict = {
    'data_id' : str,
    'event_id_no_cnty' : str,
    'inter1' : str,
    'inter2' : str,
    'interaction' : str
}

eth = eth.astype(convert_dict)
ukr = ukr.astype(convert_dict)

# Exploratory Analysis

### Basic Fact-finding - Ukraine vs. Ethiopia

In [21]:
eth.shape

(3566, 31)

In [22]:
eth.describe()

,iso,time_precision,latitude,longitude,geo_precision,fatalities,timestamp
count,3566.0,3566.000000,3566.000000,3566.000000,3566.000000,3566.000000,3.566000e+03
mean,231.0,1.535614,11.032138,38.463511,1.566181,5.090297,1.644033e+09
std,0.0,0.766584,2.431685,1.622998,0.521052,19.436587,1.324255e+07
min,231.0,1.000000,3.539100,33.205100,1.000000,0.000000,1.610410e+09
25%,231.0,1.000000,9.312200,37.483300,1.000000,0.000000,1.639526e+09
50%,231.0,1.000000,11.267000,38.757300,2.000000,0.000000,1.639586e+09
75%,231.0,2.000000,13.322600,39.591600,2.000000,3.000000,1.653948e+09
max,231.0,3.000000,14.695300,45.340800,3.000000,600.000000,1.667851e+09


In [25]:
nyt.dtypes

headline                 object
date             datetime64[ns]
doc_type                 object
material_type            object
section                 float64
keywords                 object
dtype: object

In [23]:
ukr.shape

(38507, 31)

In [24]:
ukr.describe()

,iso,time_precision,latitude,longitude,geo_precision,fatalities,timestamp
count,38507.0,38507.000000,38507.000000,38507.000000,38507.000000,38507.000000,3.850700e+04
mean,804.0,1.014283,48.475363,36.328016,1.639988,0.530994,1.651426e+09
std,0.0,0.140687,1.175126,2.619424,0.524776,6.410213,1.464968e+07
min,804.0,1.000000,43.389000,22.300000,1.000000,0.000000,1.618436e+09
25%,804.0,1.000000,47.697500,35.517900,1.000000,0.000000,1.646174e+09
50%,804.0,1.000000,48.338000,37.571000,2.000000,0.000000,1.656436e+09
75%,804.0,1.000000,48.946700,37.942000,2.000000,0.000000,1.663096e+09
max,804.0,3.000000,52.341300,40.132200,3.000000,485.000000,1.667941e+09


So far, it looks like there is far more data for Ukraine than there is for Ethiopia - over 10 times more, in fact. This is surprising because the Ethiopian conflict has been going on for over a year longer than the fighting in Ukraine.

In [26]:
eth_civ = eth[eth.actor2 == 'Civilians (Ethiopia)']
ukr_civ = ukr[ukr.actor2 == 'Civilians (Ukraine)']
print(
    eth_civ.shape,
    ukr_civ.shape
)
print(
    'Civilian-involved incidents comprise',(len(eth_civ)/len(eth))*100,'% of all ACLED incidents in Ethiopia.','\n',
    'Civilian-involved incidents comprise',(len(ukr_civ)/len(ukr))*100,'% of all ACLED incidents in Ukraine.'
)

(1101, 31) (2790, 31)
Civilian-involved incidents comprise 30.874929893438026 % of all ACLED incidents in Ethiopia. 
 Civilian-involved incidents comprise 7.2454358947723785 % of all ACLED incidents in Ukraine.


When we filter by events where actor2 = Civilians, i.e. civilians were victims, we can see that a far greater percent of incidents targeted noncombatants in Ethiopia than in Ukraine. 

In [27]:
eth_civ_acts = eth_civ.groupby(by = ['actor1']).count().sort_values(by = 'data_id',ascending = False)
ukr_civ_acts = ukr_civ.groupby(by = ['actor1']).count().sort_values(by = 'data_id',ascending = False)

In [28]:
print(
    'The perpetrators of the most violence against civilians in Ukraine are',ukr_civ_acts.index[0],',',ukr_civ_acts.index[1],'and',ukr_civ_acts.index[2],'.\nThe perpetrators of the most violence against civilians in Ukraine are',eth_civ_acts.index[0],',',eth_civ_acts.index[1],'and',eth_civ_acts.index[2],'.'
)

The perpetrators of the most violence against civilians in Ukraine are Military Forces of Russia (2000-) , Military Forces of Russia (2000-) Air Force and Unidentified Armed Group (Ukraine) .
The perpetrators of the most violence against civilians in Ukraine are TPLF: Tigray People's Liberation Front , Military Forces of Ethiopia (2018-) and OLF: Oromo Liberation Front (Shane Splinter Faction) .


In [29]:
eth_ints = eth.loc[eth['fatalities'] > 0,['interaction','fatalities']].groupby(by = ['interaction']).count().sort_values(by = 'fatalities',ascending = False)
ukr_ints = ukr.loc[ukr['fatalities'] > 0,['interaction','fatalities']].groupby(by = ['interaction']).count().sort_values(by = 'fatalities',ascending = False)


In [30]:
eth_ints

,fatalities
interaction,
12,465
17,238
37,212
13,200
27,129
47,115
14,54
44,47
78,34


In [31]:
ukr_ints

,fatalities
interaction,
78,1265
18,711
12,444
17,27
37,15
13,14
80,14
48,12
47,12


# NYT Trends

In [58]:
def containsTig(lst):
  if lst.count('Tigrayans (Ethnic Group)') > 0:
    return True
  else:
    return False

def containsEth(headline):
  words = headline.split()
  if words.count('Ethiopia') > 0:
    return True
  elif words.count('Ethiopiaâ€™s') > 0:
    return True
  else:
    return False

nyt_tig = nyt
nyt_tig['tig_tag'] = nyt_tig['keywords'].apply(containsTig)
nyt_tig['eth_head'] = nyt_tig['headline'].apply(containsEth)
nyt_tig = nyt_tig[(nyt_tig.tig_tag == True) or (nty_tig.eth_head == True)]


KeyboardInterrupt: ignored